# Data Preprocessing - House Price Dataset
## Introduction

This notebook performs **data preprocessing** for the residential housing dataset analyzed in `01_exploratory_data_analysis.ipynb`. Based on the insights from the exploratory phase, we will clean and transform the raw dataset to make it suitable for feature engineering and predictive modeling.

**Dataset:** Housing Price Prediction Data (Kaggle)

**Objective:** Produce a clean dataset stored in `data/processed/cleaned_data.csv` that can be directly used in the feature engineering stage.

**Author:** NGUYEN Ngoc Dang Nguyen – Final-year Student in Computer Science, Aix-Marseille University

**Preprocessing steps:**
1. Import libraries and load the raw data
2. Assess dataset structure and initial data quality
3. Analyze and handle missing values
4. Detect and treat outliers
5. Standardize data types
6. Create basic derived features
7. Encode categorical variables
8. Scale numerical features
9. Final data quality check
10. Save the processed dataset

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Load the raw data
df = pd.read_csv("../data/raw/housing_price_dataset.csv")
df.columns = df.columns.str.strip() 

print(f"Dataset loaded: {df.shape[0]} rows and {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")

# Keep a copy of original data for comparison
df_original = df.copy()

print(f"\nSample of the data:")
df.head()

## 2. Dataset Overview & Initial Assessment

In [ ]:
print("Initial Data Assessment")
print("=" * 40)

# Basic info
print(f"Dataset dimensions: {df.shape}")
print(f"Data types:\n{df.dtypes.value_counts()}")

# Memory and data quality check
print(f"\nData Quality Check:")
print(f"Total cells: {df.shape[0] * df.shape[1]:,}")
print(f"Missing values: {df.isnull().sum().sum():,} ({df.isnull().sum().sum()/(df.shape[0] * df.shape[1])*100:.1f}%)")
print(f"Duplicate rows: {df.duplicated().sum():,}")

# Quick data overview
df.info()

## 3. Missing Values Analysis & Handling

In [ ]:
print("\nMISSING VALUES HANDLING")
print("=" * 40)

# Calculate missing values
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

if len(missing_summary) > 0:
    print(f"Found {len(missing_summary)} columns with missing values:")
    print(missing_summary.round(2).to_string(index=False))
    
    # Visualize missing values
    plt.figure(figsize=(10, 6))
    sns.barplot(data=missing_summary.head(10), y='Column', x='Missing_Percentage')
    plt.title('Missing Values by Column')
    plt.xlabel('Percentage Missing (%)')
    plt.tight_layout()
    plt.show()
    
    # Handle missing values column by column
    print(f"\nHandling missing values:")
    
    for _, row in missing_summary.iterrows():
        col = row['Column']
        missing_pct = row['Missing_Percentage']
        
        if df[col].dtype in ['object']:
            # Categorical columns - fill with mode or 'Unknown'
            if missing_pct < 50:  # If less than 50% missing, use mode
                mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
                df[col] = df[col].fillna(mode_val)
                print(f"  * {col}: filled {row['Missing_Count']} missing values with '{mode_val}'")
            else:
                df[col] = df[col].fillna('Unknown')
                print(f"  * {col}: filled {row['Missing_Count']} missing values with 'Unknown'")
        
        else:
            # Numerical columns - fill with median or mean
            if missing_pct < 50:
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
                print(f"  {col}: filled {row['Missing_Count']} missing values with median ({median_val:.2f})")
            else:
                # Too many missing values - consider dropping the column
                print(f"  {col}: {missing_pct:.1f}% missing - consider dropping this column")

else:
    print("No missing values found!")

# Verify missing values are handled
remaining_missing = df.isnull().sum().sum()
print(f"\nMissing values after cleaning: {remaining_missing}")

## 4. Outlier Detection & Treatment

In [ ]:
print("\nOUTLIER DETECTION & TREATMENT")
print("=" * 40)

# Find numerical columns (likely price column)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numerical columns to check: {numeric_cols}")

# Find price column
price_col = None
for col in ['price', 'Price', 'price_per_m2', 'sale_price']:
    if col in df.columns:
        price_col = col
        break

if price_col is None and numeric_cols:
    price_col = numeric_cols[0]  # Use first numeric column

if price_col:
    print(f"\nAnalyzing outliers in '{price_col}':")
    
    # Calculate IQR bounds
    Q1 = df[price_col].quantile(0.25)
    Q3 = df[price_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find outliers
    outliers_mask = (df[price_col] < lower_bound) | (df[price_col] > upper_bound)
    outliers_count = outliers_mask.sum()
    
    print(f"Price statistics:")
    print(f"  * Q1 (25%): {Q1:,.2f}")
    print(f"  * Q3 (75%): {Q3:,.2f}")
    print(f"  * IQR: {IQR:,.2f}")
    print(f"  * Lower bound: {lower_bound:,.2f}")
    print(f"  * Upper bound: {upper_bound:,.2f}")
    print(f"  * Outliers found: {outliers_count} ({outliers_count/len(df)*100:.1f}%)")
    
    # Visualize outliers
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.boxplot(df[price_col])
    plt.title(f'{price_col} - Before Outlier Treatment')
    plt.ylabel(price_col)
    
    plt.subplot(1, 3, 2)
    plt.hist(df[price_col], bins=50, alpha=0.7, edgecolor='black')
    plt.axvline(lower_bound, color='red', linestyle='--', label='Lower bound')
    plt.axvline(upper_bound, color='red', linestyle='--', label='Upper bound')
    plt.title('Price Distribution with Outlier Bounds')
    plt.xlabel(price_col)
    plt.legend()
    
    # Decision on outlier treatment
    if outliers_count > len(df) * 0.1:  # More than 10% outliers
        print(f"  Too many outliers ({outliers_count/len(df)*100:.1f}%) - keeping them")
        plt.subplot(1, 3, 3)
        plt.text(0.5, 0.5, f'Keeping outliers\n({outliers_count} values)\nToo many to remove', 
                 ha='center', va='center', transform=plt.gca().transAxes, fontsize=10)
        plt.title('Decision: Keep Outliers')
    else:
        # Remove extreme outliers (optional - comment out if you want to keep them)
        print(f"  Removing {outliers_count} extreme outliers")
        df = df[~outliers_mask].copy()
        
        plt.subplot(1, 3, 3)
        plt.boxplot(df[price_col])
        plt.title(f'{price_col} - After Outlier Removal')
        plt.ylabel(price_col)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Dataset size after outlier treatment: {df.shape}")

## 5. Data Type Standardization

In [ ]:
print("\nDATA TYPE STANDARDIZATION")
print("=" * 40)

print("Current data types:")
print(df.dtypes)

# Convert obvious numeric columns that might be stored as strings
for col in df.columns:
    if df[col].dtype == 'object':
        # Try to convert to numeric if it looks like numbers
        if df[col].str.contains(r'^[\d\.\,\-\+]+$', regex=True, na=False).any():
            try:
                # Remove commas and convert to numeric
                df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='ignore')
                print(f"  Converted {col} to numeric")
            except:
                print(f"  Could not convert {col} to numeric")

# Check for date columns
date_keywords = ['date', 'year', 'time']
for col in df.columns:
    if any(keyword in col.lower() for keyword in date_keywords):
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])
                print(f"  Converted {col} to datetime")
            except:
                print(f"  Could not convert {col} to datetime")

print(f"\nData types after standardization:")
print(df.dtypes.value_counts())

## 6. Feature Creation & Engineering

In [ ]:
print("\nBASIC FEATURE CREATION")
print("=" * 40)

# Create some simple derived features based on common house price factors
features_created = []

# Example feature engineering (adjust based on your actual columns)
if 'bedrooms' in df.columns and 'bathrooms' in df.columns:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    features_created.append('total_rooms')

if 'sqft_living' in df.columns and price_col in df.columns:
    df['price_per_sqft'] = df[price_col] / df['sqft_living']
    features_created.append('price_per_sqft')

if 'yr_built' in df.columns:
    df['house_age'] = 2024 - df['yr_built']
    features_created.append('house_age')

if 'sqft_lot' in df.columns and 'sqft_living' in df.columns:
    df['lot_to_living_ratio'] = df['sqft_lot'] / df['sqft_living']
    features_created.append('lot_to_living_ratio')

if features_created:
    print(f"Created {len(features_created)} new features:")
    for feature in features_created:
        print(f"  * {feature}")
else:
    print("No additional features created (depends on available columns)")

print(f"\nDataset now has {df.shape[1]} columns")

## 7. Categorical Variable Encoding

In [ ]:
print("\nCATEGORICAL VARIABLE ENCODING")
print("=" * 40)

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns found: {categorical_cols}")

# Store encoders for later use
encoders = {}

for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"\nEncoding '{col}' ({unique_count} unique values):")
    
    if unique_count == 2:
        # Binary encoding for binary categorical variables
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        encoders[col] = le
        print(f"  Applied Label Encoding → {col}_encoded")
        
    elif unique_count <= 10:
        # One-hot encoding for low cardinality
        dummies = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df, dummies], axis=1)
        print(f"Applied One-Hot Encoding → {dummies.columns.tolist()}")
        
    else:
        # Label encoding for high cardinality
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        encoders[col] = le
        print(f"Applied Label Encoding → {col}_encoded (high cardinality)")

print(f"\nDataset after encoding: {df.shape}")

## 8. Feature Scaling Preparation

In [ ]:
print("\nFEATURE SCALING PREPARATION")
print("=" * 40)

# Identify numerical columns for scaling
numeric_cols_for_scaling = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove encoded categorical columns from scaling (they're already 0/1)
encoded_cols = [col for col in df.columns if col.endswith('_encoded') or 
                any(cat_col in col for cat_col in categorical_cols)]
numeric_cols_for_scaling = [col for col in numeric_cols_for_scaling if col not in encoded_cols]

print(f"Columns that will need scaling for ML models: {len(numeric_cols_for_scaling)}")
print(f"Columns: {numeric_cols_for_scaling}")

# Create scaled versions but keep original columns too
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[numeric_cols_for_scaling])

# Add scaled columns with '_scaled' suffix
scaled_df = pd.DataFrame(scaled_data, 
                        columns=[f"{col}_scaled" for col in numeric_cols_for_scaling],
                        index=df.index)

df = pd.concat([df, scaled_df], axis=1)
print(f"Added {len(numeric_cols_for_scaling)} scaled features")

# Store scaler for later use
import joblib
joblib.dump(scaler, '../models/scaler.pkl')
print("Scaler saved to '../models/scaler.pkl'")

## 9. Data Quality Final Check

In [ ]:
print("\nFINAL DATA QUALITY CHECK")
print("=" * 40)

print(f"Final dataset summary:")
print(f"  * Rows: {df.shape[0]:,}")
print(f"  * Columns: {df.shape[1]:,}")
print(f"  * Missing values: {df.isnull().sum().sum()}")
print(f"  * Duplicate rows: {df.duplicated().sum()}")
print(f"  * Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")

print(f"\nColumn types summary:")
print(df.dtypes.value_counts())

print(f"\nSample of processed data:")
df.head()


## 10. Save Processed Dataset

In [ ]:
print("\nSAVING PROCESSED DATASET")
print("=" * 40)

# Create processed data directory if it doesn't exist
import os
os.makedirs('../data/processed', exist_ok=True)

# Save the cleaned dataset
output_path = '../data/processed/cleaned_data.csv'
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")
print(f"Saved dataset: {df.shape[0]} rows × {df.shape[1]} columns")

# Also save a summary of preprocessing steps
preprocessing_summary = {
    'original_shape': df_original.shape,
    'final_shape': df.shape,
    'missing_values_handled': df_original.isnull().sum().sum(),
    'outliers_removed': len(df_original) - len(df),
    'features_created': features_created,
    'categorical_columns_encoded': categorical_cols,
    'features_scaled': numeric_cols_for_scaling
}

# Save preprocessing summary
import json
with open('../data/processed/preprocessing_summary.json', 'w') as f:
    json.dump(preprocessing_summary, f, indent=2, default=str)

print(f"Preprocessing summary saved to: '../data/processed/preprocessing_summary.json'")

## Preprocessing Summary & Next Steps
The data preprocessing stage successfully addressed missing values, outliers, and inconsistent data types, resulting in a clean and reliable dataset. These efforts ensure that the data is well-prepared for advanced feature engineering and robust model development in the next phases of the pipeline.